# Picocoder
_A Raspberry Pi Pico microcode glitcher_

## Connection
We connect with the Pi Pico and setup some glitching options.

In [ ]:
%matplotlib widget

POWERSUPPLY_PORT = '/dev/ttyACM0'
GLITCHER_PORT = '/dev/ttyACM1'
GLITCHER_BAUD = 115200

import struct
import time
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from importlib import reload # for debugging

import glitch_utils
from glitch_utils import GlitchyMcGlitchFace, GlitchResult
from power_supply import PowerSupply, KA3305P
glitch_utils = reload(glitch_utils)

try:
	del ps
	del glitcher
	del gc
except:
	pass
ps = KA3305P(port=POWERSUPPLY_PORT)
ps.con()
ps.power_cycle()

glitcher = glitch_utils.GlitchyMcGlitchFace(GLITCHER_PORT, GLITCHER_BAUD)
if not glitcher.ping():
	raise Exception("Glitcher not responding")
if not glitcher.ping_target():
	raise Exception("Target not responding")

gc = glitch_utils.GlitchController(groups=[r.name for r in GlitchResult], parameters=['ext_offset', 'width', 'voltage'])
gc.set_range('ext_offset', 1, 10)
gc.set_range('width', 1, 10)
gc.set_range('voltage', 0b0110011, 0b1001011) # 1V-1.24V - See Table 6-3 in TPS65094 datasheet

def reset_target(ps: PowerSupply, glitcher: GlitchyMcGlitchFace, timeout: float = 1.5) -> bool:
	ps.power_cycle()
	return glitcher.ping_target(timeout)

## Estimating glitch external offset
We obtain an estimate (in us) of the wait time between trigger and glitch.

In [ ]:
glitcher.s.write(glitch_utils.P_CMD_ESTIMATE_OFFSET)
ret_data = glitcher.s.read(4)
ext_offset = struct.unpack("<i", ret_data)[0]
gc.set_range('ext_offset', ext_offset-20, ext_offset+10)
print(f"Estimated offset: {ext_offset}")

## Measure UART delay
This command is used to measure (externally) the time it takes for the Pico to
detect data on the UART RX pin and to start processing it. <br>
The debug pin (GPIO 16, shown blue) is toggled when the Pico detects data.

![Oscilloscope capture](../firmware/inject_v2/img/fast_response_time_busy_wait_on_UART_with_timeout.png)

In [ ]:
glitcher.s.write(glitch_utils.P_CMD_UART_DEBUG_TOGGLE)
ret_data = glitcher.s.read(4)
result = (bool)(struct.unpack("<I", ret_data)[0])
print(f"Found trigger: {result}")

## Voltage range

Find where the CPU starts to stall

In [ ]:
# glitcher.voltage = 0b0110011 # 1V
glitcher.voltage = 0b1000101 # 1.18V
glitcher.width = 100
glitcher.s.reset_input_buffer() # Clear any pending data, just in case
glitcher.s.write(glitch_utils.P_CMD_VOLT_TEST)

read_result = glitcher.s.read(4)
read_result = struct.unpack("<i", read_result)[0]
print(f"Received: {read_result}")
pt = glitcher.ping_target(0)
if not pt:
	reset_target(ps, glitcher)


In [ ]:
# NOTE This figure must be generated in a different cell than the one that calls the plot function
#	   Otherwise the plot will not be displayed until the cell is done executing (no live plot)
fig, ax = plt.subplots()
ax.yaxis.set_major_locator(MaxNLocator(integer=True)) # Number of bytes received by the glitcher
ax.xaxis.set_major_locator(MaxNLocator(integer=True)) # Voltage

In [ ]:
# def get_voltage_test_marker(result, expected_count = glitch_utils.VOLT_TEST_MSG_COUNT) -> str:
# 	if result < 0:
# 		return 'xr'
# 	elif result < expected_count:
# 		return '<y'
# 	else:
# 		return '1b'

# try:
# 	for glitch_setting in gc.rand_glitch_values():
# 		[_, _, voltage] = [*glitch_setting]
# 		glitcher.voltage = voltage

# 		glitcher.s.reset_input_buffer() # Clear any pending data, just in case
# 		glitcher.s.write(glitch_utils.P_CMD_VOLT_TEST)

# 		read_result = glitcher.s.read(4)
# 		read_result = struct.unpack("<i", read_result)[0]

# 		ax.plot(voltage, read_result, get_voltage_test_marker(read_result))
# 		fig.canvas.draw() # Guarantees live update of the plot whenever a new point is added
# except KeyboardInterrupt:
# 	pass # Gentle stop, otherwise the plot might look empty

## Glitch loop

In [ ]:
# NOTE This figure must be generated in a different cell than the one that calls the plot function
#	   Otherwise the plot will not be displayed until the cell is done executing (no live plot)
fig, ax = plt.subplots()
ax.yaxis.set_major_locator(MaxNLocator(integer=True))
ax.xaxis.set_major_locator(MaxNLocator(integer=True))

In [ ]:
try:
	for glitch_setting in gc.rand_glitch_values():
		read_result, read_data = glitcher.glitch_mul(glitch_setting, expected=0x1337)
		gc.add_result(glitch_setting, read_result)
		if read_result == GlitchResult.SUCCESS:
			print(f'Got success data: 0x{read_data.hex()}')
		if read_result == GlitchResult.WEIRD:
			print(f'Got weird response: 0x{read_data.hex()}')
		if read_result == GlitchResult.BROKEN:
			print(f'The target is in broken state: {glitch_utils.RESULT_NAMES[read_data]} (0x{int.from_bytes(read_data):x})')
		ax.plot(*glitch_setting, glitch_utils.result_to_marker(read_result))
		fig.canvas.draw() # Guarantees live update of the plot whenever a new point is added
except KeyboardInterrupt:
	pass # Gentle stop, otherwise the plot might look empty